In [2]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def generate_all_csvs():
    """
    Función que procesa los datos de fútbol y genera todos los DataFrames como archivos CSV
    en el directorio 'data/'.
    """
    
    # Crear directorio 'data' si no existe
    os.makedirs('data', exist_ok=True)
    
    print("📊 Iniciando procesamiento de datos...")
    
    # --- 1. LEER ARCHIVOS EXCEL ---
    print("📖 Leyendo archivos Excel...")
    excel_path = "AUDAX/sb_team_match_stats_2025.xlsx"
    df = pd.read_excel(excel_path)
    
    excel_path_matches = "AUDAX/sb_matches_2025.xlsx"
    df_matches = pd.read_excel(excel_path_matches)
    
    # --- 2. PREPARAR DATOS ---
    print("🔧 Preparando datos...")
    df_matches_filtered = df_matches[['match_id', 'match_date', 'competition', 'season', 'match_week',  
                                     'competition_stage', 'home_team', 'away_team']]
    df = pd.merge(df, df_matches_filtered, on='match_id', how='left')
    df.columns = df.columns.str.replace("^team_match_", "", regex=True)
    
    # --- 3. CALCULAR ÍNDICES PRINCIPALES ---
    print("⚽ Calculando índices de rendimiento...")
    
    # Goal Envolvement Index (GEI)
    df["Goal Envolvement Index"] = (
        (df["xa"] + df["key_passes"] + df["assists"]) * (10 * 0.3) / 3 +
        (df["through_balls"] + df["passes_into_box"] + df["passes_inside_box"] + df["crosses_into_box"]) * (10 * 0.2) / 4 +
        df["box_cross_ratio"] * (10 * 0.05) +
        (df["sp_xa"] + df["deep_progressions"] + df["touches_inside_box"]) * (10 * 0.15) / 3 +
        (df["xgchain"] + df["xgbuildup"]) * (10 * 0.1) / 2 +
        (df["xgchain_per_possession"] + df["xgbuildup_per_possession"]) * (10 * 0.1) / 2 +
        (df["obv_pass"] + df["obv_dribble_carry"]) * (10 * 0.05) / 2 +
        df["forward_passes"] * (10 * 0.05)
    ) / df["minutes"]
    
    # Goal Conversion Index (GCI)
    df["Goal Conversion Index"] = (
        df["goals"] * (10 * 0.3) +
        df["np_xg"] * (10 * 0.2) +
        df["np_xg_per_shot"] * (10 * 0.2) +
        df["np_shots_on_target"] * (10 * 0.1) +
        df["shot_touch_ratio"] * (10 * 0.1) +
        df["penalties_won"] * (10 * 0.05) +
        df["obv_shot"] * (10 * 0.05)
    ) / df["np_shots"]
    
    # Possession GoalChance Index (PGC)
    df["Possession GoalChance Index"] = (
        (df["key_passes"] + df["assists"] + df["xa"] + df["xgchain"]) * (10 * 0.85) / 4 +
        df["touches_inside_box"] * (10 * 0.15)
    ) / df["possession"]
    
    # --- 4. NORMALIZAR ÍNDICES ---
    print("📊 Normalizando índices...")
    kpi_columns = ["Goal Envolvement Index", "Goal Conversion Index", "Possession GoalChance Index"]
    df[kpi_columns] = df[kpi_columns].fillna(0.01)
    
    scaler = MinMaxScaler(feature_range=(0.5, 9.5))
    df[[col + " (norm)" for col in kpi_columns]] = scaler.fit_transform(df[kpi_columns])
    
    # Goal Performance Index
    df["Goal Performance Index"] = ((
        df["Goal Envolvement Index (norm)"] * 3 +
        df["Goal Conversion Index (norm)"] * 4.5 +
        df["Possession GoalChance Index (norm)"] * 2
    ) / (9.5) + 3)
    df["Goal Performance Index"] = df["Goal Performance Index"].clip(upper=9.75)
    
    # --- 5. CREAR PROMEDIOS POR EQUIPO ---
    print("📈 Calculando promedios por equipo...")
    avg_kpis = df.groupby(["team_name", "team_id"])[[
        "np_xg", "np_shots", "obv_shot", "xgchain", "goals",
        "Goal Envolvement Index (norm)",
        "Goal Conversion Index (norm)",
        "Possession GoalChance Index (norm)",
        "Goal Performance Index"
    ]].mean().reset_index()
    
    # Añadir columnas identificadoras
    max_week = df['match_week'].dropna().max()
    avg_kpis["match_id"] = "AVG"
    avg_kpis["match_date"] = 2005
    avg_kpis["account_id"] = 7336
    avg_kpis["competition"] = "Chile - Primera División"
    avg_kpis["season"] = 2005
    avg_kpis["match_week"] = max_week
    avg_kpis["competition_stage"] = "Regular Season"
    avg_kpis["home_team"] = "AVG"
    avg_kpis["away_team"] = "AVG"
    
    # Promedio general
    all_teams_avg = avg_kpis[[
        "np_xg", "np_shots", "goals",
        "Goal Envolvement Index (norm)", "Goal Conversion Index (norm)",
        "Possession GoalChance Index (norm)", "Goal Performance Index"
    ]].mean()
    
    all_teams_avg_row = {
        "match_id": "AVG", "team_name": "ALL_TEAMS_AVG", "team_id": 1,
        "account_id": 7336, "match_date": 2005, "competition": "Chile - Primera División",
        "season": 2005, "match_week": max_week, "competition_stage": "Regular Season",
        "home_team": "AVG", "away_team": "AVG", **all_teams_avg.to_dict()
    }
    
    avg_kpis = pd.concat([avg_kpis, pd.DataFrame([all_teams_avg_row])], ignore_index=True)
    
    # --- 6. CREAR MATCH SCORES ---
    print("⚽ Creando marcadores de partidos...")
    goals_pivot = df.pivot_table(index="match_id", columns="team_name", values="goals", aggfunc="first")
    df_scores = df[["match_id", "home_team", "away_team"]].drop_duplicates()
    
    def get_score(row):
        try:
            home_goals = goals_pivot.loc[row["match_id"], row["home_team"]]
            away_goals = goals_pivot.loc[row["match_id"], row["away_team"]]
            return f'{row["home_team"]}({int(home_goals)}) - {row["away_team"]}({int(away_goals)})'
        except:
            return None
    
    df_scores["match_score"] = df_scores.apply(get_score, axis=1)
    
    # --- 7. DATAFRAME FINAL ---
    df_final = pd.concat([df[[
        "match_id", "team_name", "team_id", "account_id", "match_date", "competition", "season", "match_week",
        "competition_stage", "home_team", "away_team", "np_xg", "np_shots", "obv_shot", "xgchain", "goals",
        "Goal Envolvement Index (norm)", "Goal Conversion Index (norm)",
        "Possession GoalChance Index (norm)", "Goal Performance Index"
    ]], avg_kpis], ignore_index=True)
    
    df_final = df_final.merge(df_scores[["match_id", "match_score"]], on="match_id", how="left")
    df_final = df_final.fillna({"match_score": "AVG"})
    
    # --- 8. CREAR RANKINGS ---
    print("🏆 Creando rankings...")
    
    def create_ranking(metric_col, name_suffix):
        avg_only = df_final[
            (df_final["match_id"] == "AVG") & (df_final["team_name"] != "ALL_TEAMS_AVG")
        ].copy()
        
        avg_only["Rank (avg)"] = avg_only[metric_col].rank(ascending=False, method="min").astype(int)
        ranking = avg_only.sort_values(by="Rank (avg)").reset_index(drop=True)
        ranking["match_week"] = max_week
        
        ranking = ranking.rename(columns={
            "Goal Envolvement Index (norm)": "Goal Envolvement Index",
            "Goal Conversion Index (norm)": "Goal Conversion Index",
            "Possession GoalChance Index (norm)": "Possession GoalChance Index"
        })
        
        return ranking[[
            "Rank (avg)", "team_name", "team_id", "match_week",
            "Goal Performance Index", "Goal Envolvement Index", 
            "Goal Conversion Index", "Possession GoalChance Index"
        ]]
    
    ranking_avg_display_GCI = create_ranking("Goal Conversion Index (norm)", "GCI")
    ranking_avg_display_GEI = create_ranking("Goal Envolvement Index (norm)", "GEI")
    ranking_avg_display_PGI = create_ranking("Possession GoalChance Index (norm)", "PGI")
    ranking_avg_display_GPI = create_ranking("Goal Performance Index", "GPI")
    
    # --- 9. BALÓN PARADO ---
    print("🎯 Verificando datos de balón parado...")
    
    # Verificar si existen las columnas necesarias para balón parado
    setpiece_columns = [
        'team_season_shots_from_corners_pg', 'team_season_corners_pg',
        'team_season_goals_from_corners_pg', 'team_season_corner_xg_pg',
        'team_season_shots_from_free_kicks_pg', 'team_season_free_kicks_pg',
        'team_season_goals_from_free_kicks_pg', 'team_season_free_kick_xg_pg',
        'team_season_direct_free_kick_goals_pg', 'team_season_direct_free_kicks_pg',
        'team_season_direct_free_kick_xg_pg', 'team_season_shots_from_direct_free_kicks_pg',
        'team_season_shots_from_throw_ins_pg', 'team_season_throw_ins_pg',
        'team_season_goals_from_throw_ins_pg', 'team_season_throw_in_xg_pg',
        'team_season_sp_goals_pg', 'team_season_sp_pg', 'team_season_sp_goal_ratio',
        'team_season_xg_per_sp', 'team_season_sp_shot_ratio'
    ]
    
    missing_setpiece_cols = [col for col in setpiece_columns if col not in df.columns]
    
    if missing_setpiece_cols:
        print(f"⚠️  Faltan columnas de balón parado: {len(missing_setpiece_cols)} columnas")
        print("⚠️  Creando DataFrames de balón parado con valores por defecto...")
        
        # Crear DataFrames con valores por defecto para que no falle el código
        unique_teams = df[['team_name', 'team_id']].drop_duplicates()
        
        df_setpiece = unique_teams.copy()
        df_setpiece['corner Efficiency'] = 5.0
        df_setpiece['freekick Efficiency'] = 5.0
        df_setpiece['directfk Efficiency'] = 5.0
        df_setpiece['throw in Efficiency'] = 5.0
        df_setpiece['SetPiece Eficcacy Index'] = 5.0
        
        df_setpiece_efficiency = unique_teams.copy()
        df_setpiece_efficiency['GoalSetPiece Performance Index'] = 5.0
        
        print("✅ DataFrames de balón parado creados con valores por defecto (5.0)")
        
    else:
        print("✅ Procesando datos de balón parado...")
        
        def normalize_series_min_max(s, new_min=0.5, new_max=9.5):
            old_min = s.min()
            old_max = s.max()
            if old_max == old_min:
                return pd.Series(np.full_like(s, (new_min + new_max) / 2), index=s.index)
            normalized = (s - old_min) / (old_max - old_min)
            scaled = normalized * (new_max - new_min) + new_min
            return scaled
        
        # Calcular eficiencias
        df['corner_shot_efficiency'] = df['team_season_shots_from_corners_pg'] / df['team_season_corners_pg']
        df['corner_goal_efficiency'] = df['team_season_goals_from_corners_pg'] / df['team_season_corners_pg']
        df['corner_xg_efficiency'] = df['team_season_corner_xg_pg'] / df['team_season_corners_pg']
        
        df['free_kick_shot_efficiency'] = df['team_season_shots_from_free_kicks_pg'] / df['team_season_free_kicks_pg']
        df['free_kick_goal_efficiency'] = df['team_season_goals_from_free_kicks_pg'] / df['team_season_free_kicks_pg']
        df['free_kick_xg_efficiency'] = df['team_season_free_kick_xg_pg'] / df['team_season_free_kicks_pg']
        
        df['dfk_goal_efficiency'] = df['team_season_direct_free_kick_goals_pg'] / df['team_season_direct_free_kicks_pg']
        df['dfk_xg_efficiency'] = df['team_season_direct_free_kick_xg_pg'] / df['team_season_direct_free_kicks_pg']
        df['direct_free_kick_shot_efficiency'] = df['team_season_shots_from_direct_free_kicks_pg'] / df['team_season_direct_free_kicks_pg']
        
        df['throw_in_shot_efficiency'] = df['team_season_shots_from_throw_ins_pg'] / df['team_season_throw_ins_pg']
        df['throw_in_goal_efficiency'] = df['team_season_goals_from_throw_ins_pg'] / df['team_season_throw_ins_pg']
        df['throw_in_xg_efficiency'] = df['team_season_throw_in_xg_pg'] / df['team_season_throw_ins_pg']
        
        # Subíndices
        df['corner_subindex'] = (
            df['corner_goal_efficiency'] * 0.15 +
            df['corner_xg_efficiency'] * 0.10 +
            df['corner_shot_efficiency'] * 0.10
        )
        
        df['free_kick_subindex'] = (
            df['free_kick_goal_efficiency'] * 0.15 +
            df['free_kick_xg_efficiency'] * 0.10 +
            df['free_kick_shot_efficiency'] * 0.10
        )
        
        df['directfk_subindex'] = (
            df['dfk_goal_efficiency'] * 0.10 +
            df['dfk_xg_efficiency'] * 0.05 +
            df['direct_free_kick_shot_efficiency'] * 0.05
        )
        
        df['throw_in_subindex'] = (
            df['throw_in_goal_efficiency'] * 0.10 +
            df['throw_in_xg_efficiency'] * 0.05 +
            df['throw_in_shot_efficiency'] * 0.05
        )
        
        # Normalizar subíndices
        df['corner_subindex_norm'] = normalize_series_min_max(df['corner_subindex'], 0.5, 9.5)
        df['free_kick_subindex_norm'] = normalize_series_min_max(df['free_kick_subindex'], 0.5, 9.5)
        df['directfk_subindex_norm'] = normalize_series_min_max(df['directfk_subindex'], 0.5, 9.5)
        df['throw_in_subindex_norm'] = normalize_series_min_max(df['throw_in_subindex'], 0.5, 9.5)
        
        df_setpiece = df[['team_name', 'team_id',
                          'corner_subindex_norm',
                          'free_kick_subindex_norm',
                          'directfk_subindex_norm',
                          'throw_in_subindex_norm']].copy()
        
        df_setpiece.dropna(inplace=True)
        
        df_setpiece['SetPiece Eficcacy Index'] = (
            df_setpiece['corner_subindex_norm'] * 0.50 +
            df_setpiece['free_kick_subindex_norm'] * 0.25 +
            df_setpiece['directfk_subindex_norm'] * 0.15 +
            df_setpiece['throw_in_subindex_norm'] * 0.10
        )
        
        df_setpiece.rename(columns=lambda x: x.replace('_norm', ''), inplace=True)
        df_setpiece.rename(columns={
            "corner_subindex": "corner Efficiency",
            "free_kick_subindex": "freekick Efficiency",
            "directfk_subindex": "directfk Efficiency",
            "throw_in_subindex": "throw in Efficiency"
        }, inplace=True)
        
        # GoalSetPiece Performance Index
        volume_efficacy = df["team_season_sp_goals_pg"] / df["team_season_sp_pg"]
        
        def normalize(s):
            return (s - s.min()) / (s.max() - s.min())
        
        goal_conversion_norm = normalize(df["team_season_sp_goal_ratio"])
        xg_efficiency_norm = normalize(df["team_season_xg_per_sp"])
        shot_conversion_norm = normalize(df["team_season_sp_shot_ratio"])
        volume_efficacy_norm = normalize(volume_efficacy)
        
        df["GoalSetPiece Performance Index"] = (
            0.35 * goal_conversion_norm +
            0.25 * xg_efficiency_norm +
            0.20 * shot_conversion_norm +
            0.20 * volume_efficacy_norm
        ) * 9.5 + 0.5
        
        df_setpiece_efficiency = df[["team_name", "team_id", "GoalSetPiece Performance Index"]].copy()
        df_setpiece_efficiency = df_setpiece.merge(
            df_setpiece_efficiency.drop(columns=["team_name"]),
            on='team_id',
            how='inner'
        )
    
    # --- 10. DATAFRAME FINAL CON KPIS ---
    print("🔀 Combinando todos los KPIs...")
    
    # Verificar si df_setpiece_efficiency tiene la columna 'team_name' antes de eliminarla
    setpiece_cols_to_drop = ["team_name"] if "team_name" in df_setpiece_efficiency.columns else []
    if setpiece_cols_to_drop:
        df_setpiece_efficiency_merge = df_setpiece_efficiency.drop(columns=setpiece_cols_to_drop)
    else:
        df_setpiece_efficiency_merge = df_setpiece_efficiency
    
    df_GoalKPIs = ranking_avg_display_GPI.merge(
        df_setpiece_efficiency_merge,
        on='team_id',
        how='inner'
    )
    
    # Normalizar y renombrar
    def normalize_to_range(series, new_min=0.5, new_max=9.5):
        old_min = series.min()
        old_max = series.max()
        if old_max == old_min:
            return pd.Series([new_min] * len(series), index=series.index)
        return ((series - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min
    
    cols_to_norm = ["Goal Envolvement Index", "Goal Conversion Index", "Possession GoalChance Index"]
    
    for col in cols_to_norm:
        if col in df_GoalKPIs.columns:
            df_GoalKPIs[col] = normalize_to_range(df_GoalKPIs[col], 0.500, 9.500)
    
    # --- 11. TOP VALUES ---
    print("⭐ Calculando valores TOP...")
    
    # Verificar qué columnas de KPIs existen realmente en df_GoalKPIs
    possible_kpis = [
        "Goal Performance Index", "Goal Envolvement Index",
        "Goal Conversion Index", "Possession GoalChance Index",
        "corner Efficiency", "freekick Efficiency",
        "directfk Efficiency", "throw in Efficiency",
        "SetPiece Eficcacy Index", "GoalSetPiece Performance Index"
    ]
    
    # Filtrar solo las columnas que realmente existen
    kpis = [kpi for kpi in possible_kpis if kpi in df_GoalKPIs.columns]
    
    if not kpis:
        print("⚠️  No se encontraron columnas de KPIs válidas")
        # Crear DataFrame vacío como fallback
        df_KPIs_TopValues = pd.DataFrame({
            "team_name": ["TopValues (min)", "TopValues (max)"],
        })
        df_GoalKPIs_TopValues = df_GoalKPIs.copy()
    else:
        df_KPIs_TopValues = pd.DataFrame({
            "team_name": ["TopValues (min)", "TopValues (max)"],
        }, index=["TopValues (min)", "TopValues (max)"])
        
        for idx_label in ["TopValues (min)", "TopValues (max)"]:
            row = {}
            for kpi in kpis:
                try:
                    top7 = df_GoalKPIs.nlargest(7, columns=kpi)[kpi]
                    val = top7.min() if idx_label.endswith("(min)") else top7.max()
                    row[kpi] = round(val, 3)
                except:
                    row[kpi] = 5.0  # Valor por defecto en caso de error
            df_KPIs_TopValues.loc[idx_label, kpis] = pd.Series(row)
        
        df_KPIs_TopValues = df_KPIs_TopValues.reset_index(drop=True)
        
        # --- 12. DATAFRAME FINAL CON TOP VALUES ---
        if "team_id" in df_GoalKPIs.columns:
            df_GoalKPIs = df_GoalKPIs.drop(columns=["team_id"])
        if "team_id" in df_KPIs_TopValues.columns:
            df_KPIs_TopValues = df_KPIs_TopValues.drop(columns=["team_id"])
        
        match_week_val = df_GoalKPIs["match_week"].iloc[0]
        df_KPIs_TopValues["match_week"] = match_week_val
        df_KPIs_TopValues["Rank (avg)"] = 0
        
        # Asegurar que ambos DataFrames tengan las mismas columnas
        cols = df_GoalKPIs.columns.tolist()
        missing_cols_in_topvalues = [col for col in cols if col not in df_KPIs_TopValues.columns]
        for col in missing_cols_in_topvalues:
            df_KPIs_TopValues[col] = 5.0  # Valor por defecto
        
        df_KPIs_TopValues = df_KPIs_TopValues[cols]
        
        df_GoalKPIs_TopValues = pd.concat([df_KPIs_TopValues, df_GoalKPIs], ignore_index=True).round(2)
    
    # --- 13. GUARDAR TODOS LOS CSVs ---
    print("💾 Guardando archivos CSV...")
    
    datasets = {
        'df_final.csv': df_final,
        'ranking_GCI.csv': ranking_avg_display_GCI,
        'ranking_GEI.csv': ranking_avg_display_GEI,
        'ranking_PGI.csv': ranking_avg_display_PGI,
        'ranking_GPI.csv': ranking_avg_display_GPI,
        'setpiece_data.csv': df_setpiece,
        'setpiece_efficiency.csv': df_setpiece_efficiency,
        'goal_kpis.csv': df_GoalKPIs,
        'top_values.csv': df_KPIs_TopValues,
        'goal_kpis_with_top_values.csv': df_GoalKPIs_TopValues
    }
    
    for filename, dataframe in datasets.items():
        filepath = os.path.join('data', filename)
        dataframe.to_csv(filepath, index=False)
        print(f"✅ Guardado: {filepath}")
    
    print(f"\n🎉 ¡Proceso completado! Se generaron {len(datasets)} archivos CSV en el directorio 'data/'")
    print("\n📋 Archivos generados:")
    for filename in datasets.keys():
        print(f"   • {filename}")
    
    return datasets

# Ejemplo de uso:
if __name__ == "__main__":
    # Para ejecutar la función:
    generated_data = generate_all_csvs()
    print("\n✨ Todos los DataFrames han sido generados y guardados exitosamente!")

📊 Iniciando procesamiento de datos...
📖 Leyendo archivos Excel...
🔧 Preparando datos...
⚽ Calculando índices de rendimiento...
📊 Normalizando índices...
📈 Calculando promedios por equipo...
⚽ Creando marcadores de partidos...
🏆 Creando rankings...
🎯 Verificando datos de balón parado...
⚠️  Faltan columnas de balón parado: 21 columnas
⚠️  Creando DataFrames de balón parado con valores por defecto...
✅ DataFrames de balón parado creados con valores por defecto (5.0)
🔀 Combinando todos los KPIs...
⭐ Calculando valores TOP...
💾 Guardando archivos CSV...
✅ Guardado: data/df_final.csv
✅ Guardado: data/ranking_GCI.csv
✅ Guardado: data/ranking_GEI.csv
✅ Guardado: data/ranking_PGI.csv
✅ Guardado: data/ranking_GPI.csv
✅ Guardado: data/setpiece_data.csv
✅ Guardado: data/setpiece_efficiency.csv
✅ Guardado: data/goal_kpis.csv
✅ Guardado: data/top_values.csv
✅ Guardado: data/goal_kpis_with_top_values.csv

🎉 ¡Proceso completado! Se generaron 10 archivos CSV en el directorio 'data/'

📋 Archivos genera